In [254]:
import sys
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm

import torch
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold
from sklearn.metrics import log_loss, roc_auc_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

pd.set_option('display.max_columns',1000)

# ====================================================
# Configurations
# ====================================================
class CFG:
    VER = 4
    AUTHOR = 'naokisusami'
    COMPETITION = 'FDUA2'
    DATA_PATH = Path('/data')
    OOF_DATA_PATH = Path('/oof')
    MODEL_DATA_PATH = Path('/models')
    SUB_DATA_PATH = Path('/submission')
    METHOD_LIST = ['lightgbm', 'xgboost', 'catboost']
    seed = 42
    n_folds = 7
    target_col = 'MIS_Status'
    metric = 'f1_score'
    metric_maximize_flag = True
    num_boost_round = 500
    early_stopping_round = 200
    verbose = 25
    classification_lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'seed': seed,
    }
    classification_xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': 0.05,
        'random_state': seed,
    }

    classification_cat_params = {
        'learning_rate': 0.05,
        'iterations': num_boost_round,
        'random_seed': seed,
    }
    model_weight_dict = {'lightgbm': 0.50, 'xgboost': 0.10, 'catboost': 0.40}
    

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG.seed)


# ====================================================
# Metric
# ====================================================
# f1_score

# ====================================================
# LightGBM Metric
# ====================================================
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro'), CFG.metric_maximize_flag

# ====================================================
# XGBoost Metric
# ====================================================
def xgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro')


In [255]:
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
#特徴量分類
default_numerical_features = ['Term', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'ApprovalFY']
default_categorical_features = ['NewExist', 'FranchiseCode', 'RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'BankState', 'City', 'Sector']
add_numerical_features = ['FranchiseCode_count_encoding', 'RevLineCr_count_encoding', 'LowDoc_count_encoding',
                          'UrbanRural_count_encoding', 'State_count_encoding', 'BankState_count_encoding',
                          'City_count_encoding', 'Sector_count_encoding']
numerical_features = add_numerical_features + default_numerical_features 
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector']
features = numerical_features + categorical_features

In [256]:
#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        output_df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            output_df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            output_df[col] = input_df[col].fillna('50-NaN-50')
        
        return output_df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        output_df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            output_df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return output_df
    output_df = deal_missing(input_df)
    output_df = clean_money(output_df)
    output_df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'Nan':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']
        df['ApprovalTerm'] = 15 - df['ApprovalFY']
        df['DisbursementTerm'] = 15 - df['DisbursementYear']
        
        #Bankraptdataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}
        
        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = k[-26]*2
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)
        
        #State関係の特徴量作成
        StateList = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
                      'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX',
                      'UT','VT','VA','WA','WV','WI','WY']
        
        UnemploymentList = [2.6,3.7,4.0,3.4,4.1,2.8,4.0,4.6,4.2,2.7,3.1,3.7,2.8,4.6,3.1,3.0,2.9,3.9,3.5,3.1,3.0,3.7,4.3,2.9,4.0,
                          2.7,2.6,2.7,5.5,2.9,3.3,3.5,4.1,3.8,2.1,4.1,3.2,4.8,4.3,3.2,3.3,2.2,3.5,3.8,2.4,3.0,3.1
                            ,4.5,4.1,3.0,3.9]
        
        GDPList = [29603,44807,33655,27781,42376,40805,51911,56496,126421,33417,35265,38850,29843,39568,32724,35814,34770,30364,35181,
                   30282,39596,47351,32846,41353,24477,32590,28201,37075,40210,37375,45052,30943,49038,37053,34694,34040,29470,
                   38339,35153,36543,28894,35596,33742,37793,32774,34197,41617,40361,24929,34890,40303]
        
        GDPperPersonList = [37282,71008,48148,35674,53525,54943,63504,76720,164002,45958,48434,50788,39529,49083,40529,44091,43633,
                       38148,48366,37734,50729,55364,38433,51829,31127,41012,37966,46803,63662,46400,55320,41878,58126,49625,43172,
                       41073,40376,46248,43246,44738,38093,44955,42865,54766,47313,40312,54102,52810,31914,43309,63822]
        
        AveSalaryList = [40.46,50.81,45.40,37.79,56.10,49.79,60.14,49.66,79.85,43.66,46.17,44.09,36.45,51.71,40.97,38.39,40.96,
                        39.54,43.15,39.06,54.28,58.62,45.19,46.99,35.95,42.58,35.81,39.87,44.38,46.38,56.72,40.91,61.04,43.11,41.12,
                        43.45,40.75,43.46,46.10,46.38,39.63,35.00,41.88,48.35,41.11,39.54,52.07,51.04,38.48,41.46,44.03]
        
        Unemploymentdict = dict(zip(StateList,UnemploymentList))
        GDPdict = dict(zip(StateList,GDPList))
        GDPperPersondict = dict(zip(StateList,GDPperPersonList))
        AveSalarydict = dict(zip(StateList,AveSalaryList))
        
        df['Unemployment_By_State'] = df['State'].map(Unemploymentdict)
        df['GDP_By_State'] = df['State'].map(GDPdict)
        df['GDPperPerson_By_State'] = df['State'].map(GDPperPersondict)
        df['AveSalary_By_State'] = df['State'].map(AveSalarydict)
        
        #df['State-City'] = df['State']+df['City']
        
        return df
    output_df = make_features(output_df)
    return output_df

In [257]:
#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)
print(train_df)

       Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode RevLineCr  \
0       163     21         1          0            0              1         N   
1        84      6         1          4            0              0         0   
2       242     45         1          4           90              0         N   
3       237      4         1          0            0              0         N   
4       184      0         1          0            0              0         N   
...     ...    ...       ...        ...          ...            ...       ...   
42302   283     14         1          0            0              1         N   
42303    53      2         1          0            0              0         Y   
42304    59      6         0          0            0              1         N   
42305   295     18         1          0            8              0         N   
42306    84      4         1          0            8              0         N   

      LowDoc DisbursementDa

（以下はPreprocessingに本来組み込むべきだが，コードが煩雑になるので，いったん切り出している．）

In [258]:
#カウントエンコーディング
for col in ['FranchiseCode', 'RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'BankState', 'City', 'Sector']:
    count_dict = dict(train_df[col].value_counts())
    train_df[f'{col}_count_encoding'] = train_df[col].map(count_dict)
    test_df[f'{col}_count_encoding'] = test_df[col].map(count_dict).fillna(1).astype(int)
#ラベルエンコーディング
for col in categorical_features:
    encoder = LabelEncoder()
    encoder.fit(train_df[col])
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])
    
print(train_df)
    

       Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode  \
0       163     21         1          0            0              1   
1        84      6         1          4            0              0   
2       242     45         1          4           90              0   
3       237      4         1          0            0              0   
4       184      0         1          0            0              0   
...     ...    ...       ...        ...          ...            ...   
42302   283     14         1          0            0              1   
42303    53      2         1          0            0              0   
42304    59      6         0          0            0              1   
42305   295     18         1          0            8              0   
42306    84      4         1          0            8              0   

       RevLineCr  LowDoc DisbursementDate  MIS_Status  Sector ApprovalDate  \
0              1       3        31-Jan-98           1       0    22-S

In [259]:

#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features = ['RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0', 'LowDoc_0.0','LowDoc_1.0',
                        'LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','UrbanRural', 'State', 'Sector']
RemoveList=['DisbursementDate','City','ApprovalDate','ApprovalFY','BankState','FranchiseCode']
features = train_df.columns.tolist()
for i in RemoveList:
    features.remove(i)
features.remove('MIS_Status')

for i in RemoveList:
    train_df.drop([i],axis=1)
    test_df.drop([i],axis=1)

In [260]:
print(train_df)
print(features)

       Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode  \
0       163     21         1          0            0              1   
1        84      6         1          4            0              0   
2       242     45         1          4           90              0   
3       237      4         1          0            0              0   
4       184      0         1          0            0              0   
...     ...    ...       ...        ...          ...            ...   
42302   283     14         1          0            0              1   
42303    53      2         1          0            0              0   
42304    59      6         0          0            0              1   
42305   295     18         1          0            8              0   
42306    84      4         1          0            8              0   

       RevLineCr_1.0  RevLineCr_0.0  RevLineCr_4.0  RevLineCr_3.0  \
0                  1              0              0              0   
1        

In [261]:
#lightgbmでの学習メソッドの定義
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
    lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
    model = lgb.train(
                params = CFG.classification_lgb_params,
                train_set = lgb_train,
                num_boost_round = CFG.num_boost_round,
                valid_sets = [lgb_train, lgb_valid],
                feval = lgb_metric,
                callbacks=[lgb.early_stopping(stopping_rounds=CFG.early_stopping_round,
                                              verbose=CFG.verbose)]
            )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

#xgboostでの学習メソッドの定義
def xgboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    xgb_train = xgb.DMatrix(data=x_train, label=y_train)
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid)
    model = xgb.train(
                CFG.classification_xgb_params,
                dtrain = xgb_train,
                num_boost_round = CFG.num_boost_round,
                evals = [(xgb_train, 'train'), (xgb_valid, 'eval')],
                early_stopping_rounds = CFG.early_stopping_round,
                verbose_eval = CFG.verbose,
                feval = xgb_metric,
                maximize = CFG.metric_maximize_flag,
            )
    # Predict validation
    valid_pred = model.predict(xgb.DMatrix(x_valid))
    return model, valid_pred

#catboostでの学習メソッドの定義
def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    cat_train = Pool(data=x_train, label=y_train, cat_features=categorical_features)
    cat_valid = Pool(data=x_valid, label=y_valid, cat_features=categorical_features)
    model = CatBoostClassifier(**CFG.classification_cat_params)
    model.fit(cat_train,
              eval_set = [cat_valid],
              early_stopping_rounds = CFG.early_stopping_round,
              verbose = CFG.verbose,
              use_best_model = True)
    # Predict validation
    valid_pred = model.predict_proba(x_valid)[:, 1]
    return model, valid_pred
#任意のモデルでのクロスバリデーション学習メソッドの定義
def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df)):
        print('-'*50)
        print(f'{method} training fold {fold+1}')

        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'xgboost':
            model, valid_pred = xgboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)

        # Save best model
        pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = f1_score(train_df[CFG.target_col], oof_predictions >= 0.5, average='macro')
    print(f'{method} our out of folds CV f1score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    oof_df.to_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv', index = False)
#学習メソッドの定義
def Learning(input_df: pd.DataFrame, features: list, categorical_features: list):
    for method in CFG.METHOD_LIST:
        gradient_boosting_model_cv_training(method, input_df, features, categorical_features)

In [262]:
Learning(train_df, features, categorical_features)

--------------------------------------------------
lightgbm training fold 1
[LightGBM] [Info] Number of positive: 32336, number of negative: 3927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2263
[LightGBM] [Info] Number of data points in the train set: 36263, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.891708 -> initscore=2.108305
[LightGBM] [Info] Start training from score 2.108305
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[175]	training's auc: 0.911817	training's f1score: 0.683331	valid_1's auc: 0.76074	valid_1's f1score: 0.638753
--------------------------------------------------
lightgbm training fold 2
[LightGBM] [Info] Number of positive: 32389, number of negative: 3874
[LightGBM] [Inf

[75]	train-logloss:0.25934	train-f1score:0.66992	eval-logloss:0.28195	eval-f1score:0.65321
[100]	train-logloss:0.24924	train-f1score:0.68298	eval-logloss:0.27988	eval-f1score:0.65875
[125]	train-logloss:0.24261	train-f1score:0.69154	eval-logloss:0.27940	eval-f1score:0.66202
[150]	train-logloss:0.23691	train-f1score:0.70028	eval-logloss:0.27918	eval-f1score:0.66274
[175]	train-logloss:0.23250	train-f1score:0.70588	eval-logloss:0.27919	eval-f1score:0.66265
[200]	train-logloss:0.22716	train-f1score:0.71355	eval-logloss:0.27951	eval-f1score:0.66280
[215]	train-logloss:0.22421	train-f1score:0.71747	eval-logloss:0.27958	eval-f1score:0.66546
--------------------------------------------------
xgboost training fold 4
[0]	train-logloss:0.66007	train-f1score:0.09733	eval-logloss:0.65981	eval-f1score:0.09440
[25]	train-logloss:0.33508	train-f1score:0.66035	eval-logloss:0.33323	eval-f1score:0.66296
[50]	train-logloss:0.27873	train-f1score:0.66021	eval-logloss:0.28269	eval-f1score:0.66043
[75]	train

350:	learn: 0.2536077	test: 0.2757350	best: 0.2751602 (242)	total: 28.9s	remaining: 12.3s
375:	learn: 0.2518458	test: 0.2758236	best: 0.2751602 (242)	total: 31.4s	remaining: 10.4s
400:	learn: 0.2500129	test: 0.2758040	best: 0.2751602 (242)	total: 33.7s	remaining: 8.33s
425:	learn: 0.2485456	test: 0.2759629	best: 0.2751602 (242)	total: 35.9s	remaining: 6.24s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.275160201
bestIteration = 242

Shrink model to first 243 iterations.
--------------------------------------------------
catboost training fold 2
0:	learn: 0.6446960	test: 0.6450028	best: 0.6450028 (0)	total: 63.6ms	remaining: 31.7s
25:	learn: 0.3083276	test: 0.3132148	best: 0.3132148 (25)	total: 2.19s	remaining: 40s
50:	learn: 0.2840610	test: 0.2923025	best: 0.2923025 (50)	total: 4.39s	remaining: 38.7s
75:	learn: 0.2771552	test: 0.2879885	best: 0.2879885 (75)	total: 6.85s	remaining: 38.2s
100:	learn: 0.2732102	test: 0.2861170	best: 0.2861170 (100)	total: 9.2s	remai

450:	learn: 0.2480555	test: 0.2715031	best: 0.2713725 (364)	total: 41.4s	remaining: 4.5s
475:	learn: 0.2463422	test: 0.2715570	best: 0.2713725 (364)	total: 43.6s	remaining: 2.2s
499:	learn: 0.2448512	test: 0.2714896	best: 0.2713725 (364)	total: 45.5s	remaining: 0us

bestTest = 0.2713724656
bestIteration = 364

Shrink model to first 365 iterations.
--------------------------------------------------
catboost training fold 6
0:	learn: 0.6444303	test: 0.6451748	best: 0.6451748 (0)	total: 73ms	remaining: 36.4s
25:	learn: 0.3079241	test: 0.3175302	best: 0.3175302 (25)	total: 2.07s	remaining: 37.8s
50:	learn: 0.2828073	test: 0.2953008	best: 0.2953008 (50)	total: 4.11s	remaining: 36.2s
75:	learn: 0.2762748	test: 0.2905081	best: 0.2905081 (75)	total: 6.29s	remaining: 35.1s
100:	learn: 0.2724156	test: 0.2882905	best: 0.2882905 (100)	total: 8.6s	remaining: 34s
125:	learn: 0.2695168	test: 0.2873307	best: 0.2873093 (123)	total: 10.8s	remaining: 32.2s
150:	learn: 0.2672784	test: 0.2868416	best: 0.28

In [263]:
def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds
def xgboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'xgboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(xgb.DMatrix(x_test))
        test_pred += pred
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict_proba(x_test)[:, 1]
        test_pred += pred
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list, categorical_features: list):
    x_test = test_df[features]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test)
    if method == 'xgboost':
        test_pred = xgboost_inference(x_test)
    if method == 'catboost':
        test_pred = catboost_inference(x_test)
    return test_pred

def Predicting(input_df: pd.DataFrame, features: list, categorical_features: list):
    output_df = input_df.copy()
    output_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        output_df[f'{method}_pred_prob'] = gradient_boosting_model_inference(method, input_df, features, categorical_features)
        output_df['pred_prob'] += CFG.model_weight_dict[method] * output_df[f'{method}_pred_prob']
    return output_df

In [264]:
test_df = Predicting(test_df, features, categorical_features)

In [265]:
#後処理の定義
def Postprocessing(train_df: pd.DataFrame(), test_df: pd.DataFrame()) -> (pd.DataFrame(), pd.DataFrame()):
    train_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv')
        train_df['pred_prob'] += CFG.model_weight_dict[method] * oof_df[f'{method}_prediction']
    best_score = 0
    best_v = 0
    for v in tqdm(np.arange(1000) / 1000):
        score = f1_score(oof_df[CFG.target_col], train_df[f'pred_prob'] >= v, average='macro')
        if score > best_score:
            best_score = score
            best_v = v
    print(best_score, best_v)
    test_df['target'] = np.where(test_df['pred_prob'] >= best_v, 1, 0)
    return train_df, test_df

In [266]:
#後処理
train_df, test_df = Postprocessing(train_df, test_df)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.684517058855488 0.737


In [267]:
test_df[['target']].to_csv(f'seed{CFG.seed}_ver{CFG.VER}_{CFG.AUTHOR}_submission.csv', header=False)

特徴量の重要度を確認する方法

In [268]:
model = pickle.load(open(f'lightgbm_fold1_seed42_ver{CFG.VER}.pkl', 'rb'))
importance_df = pd.DataFrame(model.feature_importance(), index=features, columns=['importance'])
importance_df['importance'] = importance_df['importance'] / np.sum(importance_df['importance'])
importance_df.sort_values('importance', ascending=False)

,importance
State,0.123048
Term,0.075048
NoEmp,0.058857
City_count_encoding,0.056190
SBA_Appv,0.053714
ApprovalDay,0.049333
DisbursementGross,0.041714
Sector,0.041524
CompanyLong,0.035048
DisbursementMonth,0.034286


In [269]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42307 entries, 0 to 42306
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Term                          42307 non-null  int64  
 1   NoEmp                         42307 non-null  int64  
 2   NewExist                      42307 non-null  int32  
 3   CreateJob                     42307 non-null  int64  
 4   RetainedJob                   42307 non-null  int64  
 5   FranchiseCode                 42307 non-null  int64  
 6   RevLineCr_1.0                 42307 non-null  int64  
 7   RevLineCr_0.0                 42307 non-null  int64  
 8   RevLineCr_4.0                 42307 non-null  int64  
 9   RevLineCr_3.0                 42307 non-null  int64  
 10  RevLineCr_2.0                 42307 non-null  int64  
 11  LowDoc_3.0                    42307 non-null  int64  
 12  LowDoc_2.0                    42307 non-null  int64  
 13  L